In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "jperez2024"
password = "HotDog2024"

# Connect to database via CRUD Module
host = "localhost"
port = 27017
db_name = "AAC"
collection_name = "animals"

db = AnimalShelter(username, password, host, port, db_name, collection_name)

# Fetch all data from the database
df = pd.DataFrame.from_records(db.read({}))

# Remove the '_id' column to avoid issues with DataTable
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height':'100px'}), href='http://www.graziososalvare.com')),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H3('Developed by Jusitn Perez'))),
    html.Div(
        # Interactive filtering options will go here
        children=[
            html.Label("Select Rescue Type:"),
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water Rescue'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                    {'label': 'Reset', 'value': 'Reset'}
                ],
                value='Reset'
            ),
            html.Br(),
            dcc.Dropdown(
                id='animal-list-dropdown',
                options=[
                    {'label': 'All Animals', 'value': 'all'},
                    {'label': 'Dogs Only', 'value': 'dog'},
                    {'label': 'Cats Only', 'value': 'cat'}
                ],
                value='all',
                style={'width': '200px'}
            ),
            html.Button('Apply', id='apply-button', n_clicks=0)
        ]
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         # Set up the features for the interactive data table
                         page_size=10,
                         style_table={'overflowX': 'auto'},
                         style_cell={
                             'height': 'auto',
                             'minWidth': '0px', 'maxWidth': '180px',
                             'whiteSpace': 'normal'
                         },
                         sort_action="native",
                         filter_action="native",
                         row_selectable="single",
                         selected_rows=[],
                         style_header={
                             'backgroundColor': 'rgb(30, 30, 30)',
                             'color': 'white'
                         },
                         style_data={
                             'backgroundColor': 'rgb(50, 50, 50)',
                             'color': 'white'
                         }
                        ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display': 'flex'},
         children=[
             html.Div(
                 id='graph-id',
                 className='col s12 m6',
             ),
             html.Div(
                 id='map-id',
                 className='col s12 m6',
             )
         ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query = {}
    if filter_type == 'Water Rescue':
        query = {"$and": [{"animal_type": "Dog"}, {"$or": [{"breed": "Labrador Retriever Mix"}, {"breed": "Chesapeake Bay Retriever"}, {"breed": "Newfoundland"}]}, {"sex_upon_outcome": "Intact Female"}, {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}]}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"$and": [{"animal_type": "Dog"}, {"$or": [{"breed": "German Shepherd"}, {"breed": "Alaskan Malamute"}, {"breed": "Old English Sheepdog"}, {"breed": "Siberian Husky"}, {"breed": "Rottweiler"}]}, {"sex_upon_outcome": "Intact Male"}, {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}]}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"$and": [{"animal_type": "Dog"}, {"$or": [{"breed": "Doberman Pinscher"}, {"breed": "German Shepherd"}, {"breed": "Golden Retriever"}, {"breed": "Bloodhound"}, {"breed": "Rottweiler"}]}, {"sex_upon_outcome": "Intact Male"}, {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}]}
    df_filtered = pd.DataFrame.from_records(db.read(query))
    df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('apply-button', 'n_clicks')],
    [State('animal-list-dropdown', 'value')]
)
def update_graphs(viewData, n_clicks, animal_list):
    if viewData is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    if animal_list == 'dog':
        dff = dff[dff['animal_type'] == 'Dog']
    elif animal_list == 'cat':
        dff = dff[dff['animal_type'] == 'Cat']

    if dff.empty:
        return [html.H3("No data to display")]

    fig = px.pie(dff, names='breed', title='Preferred Animals')
    return [dcc.Graph(figure=fig)]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


OperationFailure: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}